In [1]:
# Calling the necessary libraries
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

### 1. Business Problem
FLO wants to determine a roadmap for its sales and marketing activities. In order for the company to make a medium to long term plan, it is expected to estimate the potential value that existing customers will provide to the company in the future.

### 2. Dataset Story
The data set consists of information obtained from the past shopping behavior of customers who made their last purchases as OmniChannel (both online and offline shoppers) in 2020 - 2021.

### 3. Features
+ master_id - Unique customer identifier
+ order_channel - Indication of the platform used for shopping (Android, iOS, Desktop, Mobile)
+ last_order_channel - The channel used for the most recent purchase
+ first_order_date - Date of the customer's initial purchase
+ last_order_date - Date of the customer's most recent purchase
+ last_order_date_online - Date of the customer's last online purchase
+ last_order_date_offline - Date of the customer's last offline purchase
+ order_num_total_ever_online - Total number of purchases made online by the customer
+ order_num_total_ever_offline - Total number of purchases made offline by the customer
+ customer_value_total_ever_offline - Total amount spent by the customer in offline purchases
+ customer_value_total_ever_online - Total amount spent by the customer in online purchases
+ interested_in_categories_12 - List of categories in which the customer has made purchases in the last 12 months

### 4.Databehandling

In [2]:
df_ = pd.read_csv("flo_data_20k.csv")
df = df_.copy()

In [3]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


In [4]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline', 'order_num_total_ever_online', 'order_num_total_ever_offline', 'customer_value_total_ever_offline', 'customer_value_total_ever_online', 'interested_in_categories_12'], dtype='object')

In [5]:
df.shape

(19945, 12)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [8]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [9]:
 # Functions required to suppress outliers
def outliers_threshold(dataframe,variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [10]:
def replace_with_threshold(dataframe, variable):
    low_limit, up_limit = outliers_threshold(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit, 0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)

In [11]:
# Suppression of outliers of “order_num_total_ever_online”, “order_num_total_ever_offline”,
# “customer_value_total_ever_offline”, “customer_value_total_ever_online” variables
columns = ["order_num_total_ever_online","order_num_total_ever_offline","customer_value_total_ever_offline","customer_value_total_ever_online"]

for col in columns:
    replace_with_threshold(df,col)

In [12]:
# It is stated that customers shop both online and offline. 
# Creating new variables for the total number of purchases and expenditures of each customer.
df["order_num_total"] = df["order_num_total_ever_offline"] + df["order_num_total_ever_online"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [13]:
# Converting the type of date expressing variables to date
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

#### 4.1 Creating CLTV Data Structure

In [14]:
# Determination of the day of analysis
df["last_order_date"].max()
analysis_date = dt.datetime(2021, 6, 1)

In [15]:
# Creating a new cltv dataframe with customer_id, recency_cltv_weekly, T_weekly, frequency and monetary_cltv_avg values.
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
cltv_df["T_weekly"] = (analysis_date - df["first_order_date"]).dt.days / 7
cltv_df["frequency"] = df["order_num_total"].astype(int)
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"] 

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [16]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99


#### 4.2 Establishment of BG/NBD, Gamma-Gamma Models Calculation of 6-month CLTV

In [17]:
# Establishment of BG/NBD Model
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

C:\Users\tzcnm\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [18]:
# Forecasting expected purchases from customers within 3 months 
cltv_df["expected_sales_3_month"] = bgf.predict(4*3, cltv_df["frequency"],
                                                cltv_df["recency_cltv_weekly"],
                                                cltv_df["T_weekly"])

In [19]:
# Forecasting expected purchases from customer within 6 months
cltv_df["expected_sales_6_month"] = bgf.predict(4*6, cltv_df["frequency"],
                                               cltv_df["recency_cltv_weekly"],
                                               cltv_df["T_weekly"])

In [20]:
# Establishment of Gamma-Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df["frequency"],
        cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [21]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                       cltv_df["monetary_cltv_avg"])

In [22]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,expected_sales_3_month,expected_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33


In [23]:
# Calculate the 6-month CLTV and add it to the dataframe as cltv.

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)

cltv_df["cltv"] = cltv

In [24]:
cltv_df.sort_values("cltv", ascending=False)[:5]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,expected_sales_3_month,expected_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.86,7.86,4,1401.80,1.09,2.19,1449.06,3327.78
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.14,13.14,11,758.09,1.97,3.94,767.36,3172.39
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.71,101.00,7,1106.47,0.72,1.44,1127.61,1708.98
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.29,74.57,16,501.87,1.57,3.13,506.17,1662.61
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52,166.22,4.66,9.31,166.71,1628.89


In [25]:
# Creation of Segments According to CLTV

cltv_df["segment"]= pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,expected_sales_3_month,expected_sales_6_month,exp_average_value,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5,187.87,0.97,1.95,193.63,395.73,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21,95.88,0.98,1.97,96.67,199.43,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5,117.06,0.67,1.34,120.97,170.22,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2,60.98,0.70,1.40,67.32,98.95,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2,104.99,0.40,0.79,114.33,95.01,D


In [26]:
# Calculate the mean values of recency, frequency, monetary and CLV by customer segment

cltv_df.groupby("segment")[["recency_cltv_weekly", "frequency", "monetary_cltv_avg", "cltv"]].agg("mean").sort_values(by="cltv",
                                                                                                                   ascending=False)

,recency_cltv_weekly,frequency,monetary_cltv_avg,cltv
segment,,,,
A,67.43,6.65,228.83,362.32
B,81.99,5.09,160.64,199.53
C,92.63,4.40,125.79,138.31
D,139.00,3.77,93.15,80.34
